<a href="https://colab.research.google.com/github/gauss5930/Natural-Language-Processing/blob/main/GPT-1/GPT_1%20Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT-1 Implementation

GPT-1 구현 코드는 [GPT 구현하기](https://paul-hyun.github.io/gpt-01/)를 참고하여 작성되었다.

우선 GPT를 구현하기 전에 GPT에 대해 간략하게 설명하면 GPT는 Transformer의 Decoder만을 사용한 Pre-trained LM이다.

### 1. Config

Transformer와 파라미터를 동일하게 설정하였다. GPT는 Transformer의 Decoder만을 사용하므로 Encoder 부분은 제거하고 사용하였다.

In [ ]:
config = Config({
    'n_dec_vocab': len(vocab),
    'n_dec_seq': 256,
    'n_layer': 6,
    'd_hidn': 256,
    'i_pad': 0,
    'd_ff': 1024,
    'n_head': 4,
    'd_head': 64,
    'dropout': 0.1,
    'layer_norm_epsilon': 1e-12
})
print(config)

# 2. Decoder

GPT는 Transformer의 Encoder는 사용하지 않고 Decoder만 사용하므로 Decoder에서 Encoder의 출력과 Attention을 하는 부분인 Encoder-Decoder-Multi-Head Attention 부분은 제거하고 사용하였다. 그 외에 나머지 부분은 Transformer와 동일하다.

In [ ]:
# Decoder Layer
class DecoderLayer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config

    self.self_attn = MultiHeadAttention(self.config)
    self.layer_norm1 = nn.LayerNorm(self.config.d_hidn, eps = self.config.layer_norm_epsilon)
    self.pos_ffn = PoswiseFeedForwardNet(self.config)
    self.layer_norm3 = nn.LayerNorm(self.config.d_hidn, eps = self.config.layer_norm_epsilon)

  def forward(self, dec_inputs, self_attn_mask):
    # (batch_size, n_dec_seq, d_hidn), (batch_size, n_head, n_dec_seq, n_dec_seq)
    self_att_outputs, self_attn_prob = self.self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask)
    self_att_outputs = self.layer_norm1(dec_inputs + self_att_outputs)
    # (batch_size, n_dec_seq, d_hidn)
    ffn_outputs = self.po_ffn(self_att_outputs)
    ffn_outputs = self.layer_norm3(self_att_outputs + ffn_outputs)
    # (batch_size, n_dec_seq, d_hidn), (batch_size, n_head, n_dec_seq, n_dec_seq), (batch_size, n_head, n_dec_seq, n_enc_seq)
    return ffn_outputs, self_attn_prob

In [ ]:
# Decoder
class Decoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config

    self.dec_emb = nn.Embedding(self.config.n_dec_vocab, self.config.d_hidn)
    sinusoid_table = torch.FloatTensor(det_sinusoid_encoding_table(self.config.n_dec_seq + 1, self.config.d_hidn))
    self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze = True)

    self.layers = nn.ModuleList([DecoderLayer(self.config) for _ in range(self.config.n_layer)])

  def forward(self, dec_inputs):
    positions = torch.arange(dec_inputs.size(1), device = dec_inputs.device, dtype = dec_inputs.dtype).expand(dec_inputs.size(0), dec_inputs.size(1)).contiguous() + 1
    pos_mask = dec_inputs.eq(self.config.i_pad)
    positions.masked_fill_(pos_mask, 0)

    # (batch_size, n_dec_seq, d_hidn)
    dec_outputs = self.dec_emb(dec_inputs) + self.pos_emb(positions)

    # (batch_size, n_dec_seq, n_dec_seq)
    dec_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs, self.config.i_pad)
    # (batch_size, n_dec_seq, n_dec_seq)
    dec_attn_decoder_mask = get_attn_decoder_mask(dec_inputs)
    # (batch_size, n_dec_seq, n_dec_seq)
    dec_self_attn_mask = torch.gt((dec_attn_mask + dec_attn_decoder_mask), 0)

    self_attn_probs = []
    for layer in self.layers:
      # (batch_size, n_dec_seq, d_hidn), (batch_size, n_dec_seq, n_dec_seq)
      dec_outputs, self_attn_prob = layer(dec_outputs, dec_self_attn_mask)
      self_attn_probs.append(self_attn_prob)
    # (batch_size, n_dec_seq, d_hidn), [(batch_size, n_dec_seq, n_dec_seq)]
    return dec_outputs, self_attn_probs

# 3. GPT

GPT는 단순히 Transformer Decoder를 실행
Pre-traing 모델을 저장하기 위한 save, 저장된 모델을 읽기 위한 load 함수가 추가로 정의의

In [ ]:
class GPT(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config

    self.decoder = Decoder(self.config)

  def forward(self, dec_inputs):
    # (batch_size, n_seq, d_hidn), [(batch_size, n_head, n_dec_seq, n_dec_seq)]
    dec_outputs, dec_self_attn_probs = self.decoder(dec_inputs)
    # (batch_size, n_dec_seq, n_dec_vocab), [(batch_size, n_head, n_dec_seq, n_dec_seq)]
    return dec_outputs, dec_self_attn_probs

  def save(self, epoch, loss, path):
    torch.save({
        'epoch': epoch, 
        'loss': loss, 
        'state_dict': self.state_dict()
    }, path)

  def load(self, path):
    save = torch.load(path)
    self.load_state_dict(save['state_dict'])
    return save['epoch'], save['loss']

# 4. Pre-traing Model

GPT를 pre-train 하기 위한 클래스. GPT pre-train 클래스의 목적은 입력 단어에 대한 다음 단어를 예측하는 것이다.

In [ ]:
class GPTPretraing(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config

    self.gpt = GPT(self.config)
    # 단어를 예측하기 위한 projection_lm을 선언
    self.projection_lm = nn.Linear(self.config.d_hidn, self.config.n_dec_vocab, bias = False)
    # Decoder의 Embedding & weight를 공유
    self.projection_lm.weight = self.gpt.decoder.dec_emb.weight

  def forward(self, dec_inputs):
    # (batch_size, n_dec_seq, d_hidn), [(batch_size, n_head, n_dec_seq, n_dec_seq)]
    dec_outputs, dec_self_attn_probs = self.gpt(dec_inputs)
    # (batch_size, n_dec_seq, n_dec_vocab)
    # GPT 실행 결과를 입력으로 projection_lm을 실행해서 단어를 예측측
    logits_lm = self.projection_lm(dec_outputs)
    # (batch_size, n_dec_seq - 1, n_dec_vocab), (batch_size, n_output), [(batch_size, n_head, n_dec_seq, n_dec_seq)]
    # 결과의 마지막을 제외한 나머지를 리턴
    return logits_lm[:, :-1, :].contiguous(), dec_self_attn_probs